## 3. Introduction to Ray Tune


<img src="https://docs.ray.io/en/master/_images/tune_overview.png" width="250">

Tune is a Python library for experiment execution and hyperparameter tuning at any scale.

### Getting started

We start by defining our training function

In [6]:
import ray
from ray import tune, train
from ray.tune.search import optuna
import numpy as np
from typing import Any


def my_simple_model(distance: np.ndarray, a: float) -> np.ndarray:
    return distance * a

# Step 1: Define the training function
def train_my_simple_model(config: dict[str, Any]) -> None: # Expected function signature for Ray Tune
    distances = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
    total_amts = distances * 10
    
    a = config["a"]
    predictions = my_simple_model(distances, a)
    rmse = np.sqrt(np.mean((total_amts - predictions) ** 2))

    train.report({"rmse": rmse}) # This is how we report the metric to Ray Tune

<div class="alert alert-block alert-warning">
<b>Note:</b> how the training function needs to accept a config argument. This is because Ray Tune will pass the hyperparameters to the training function as a dictionary.
</div>

Next, we define and run the hyperparameter tuning job by following these steps:

1. Create a `Tuner` object (in our case named `tuner`)
2. Call `tuner.fit`

In [7]:
# Step 2: Set up the Tuner
tuner = tune.Tuner(
    trainable=train_my_simple_model,  # Training function or class to be tuned
    param_space={
        "a": tune.randint(0, 20),  # Hyperparameter: a
    },
    tune_config=tune.TuneConfig(
        metric="rmse",  # Metric to optimize (minimize)
        mode="min",     # Minimize the metric
        num_samples=5,  # Number of samples to try
    ),
)

# Step 3: Run the Tuner and get the results
results = tuner.fit()

2024-11-29 09:13:31,837	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/ray/ray_results/train_my_simple_model_2024-11-29_09-13-26' in 0.0075s.
2024-11-29 09:13:31,841	INFO tune.py:1041 -- Total run time: 3.57 seconds (3.33 seconds for the tuning loop).


(autoscaler +4m37s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(train_pytorch pid=38448) Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
(train_pytorch pid=38448) Failed to download (trying next):
(train_pytorch pid=38448) HTTP Error 403: Forbidden
(train_pytorch pid=38448) 
(train_pytorch pid=38448) Downloading https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz
(train_pytorch pid=38448) Downloading https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz to ./data/MNIST/raw/train-images-idx3-ubyte.gz


In [12]:
# Step 4: Get the best result
best_result = results.get_best_result()
best_result

Result(
  metrics={'rmse': 1.3266499161421599},
  path='/home/ray/ray_results/train_my_simple_model_2024-11-29_09-13-26/train_my_simple_model_3207e_00000_0_a=6_2024-11-29_09-13-28',
  filesystem='local',
  checkpoint=None
)

In [13]:
best_result.config

{'a': 6}

So let's recap what actually happened here ?

```python
tuner = tune.Tuner(
    trainable=train_my_simple_model,  # Training function or class to be tuned
    param_space={
        "a": tune.randint(0, 20),  # Hyperparameter: a
    },
    tune_config=tune.TuneConfig(
        metric="rmse",  # Metric to optimize (minimize)
        mode="min",     # Minimize the metric
        num_samples=5,  # Number of samples to try
    ),
)

results = tuner.fit()
```

A Tuner accepts:
- A training function or class which is specified by `trainable`
- A search space which is specified by `param_space`
- A metric to optimize which is specified by `metric` and the direction of optimization `mode`
- `num_samples` which correlates to the number of trials to run

`tuner.fit` then runs multiple trials in parallel, each with a different set of hyperparameters, and returns the best set of hyperparameters found.
